In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
#构建模型前向传播
class soft_net(nn.Module):

    def __init__(self,num1,num2,num3,num4):
        super(soft_net,self).__init__()
        self.net1 = nn.Sequential(nn.Linear(num1,num2),
                                    nn.Sigmoid(),
                                    nn.Linear(num2,num3),
                                    nn.Sigmoid(),
                                    nn.Linear(num3,num4),
                                    nn.Sigmoid())
        
        self.net2 = nn.Sequential(nn.Linear(num1,num2),
                                    nn.Sigmoid(),
                                    nn.Linear(num2,num3),
                                    nn.Sigmoid(),
                                    nn.Linear(num3,num4),
                                    nn.Sigmoid())
    
    def forward(self,x):

        y1 = self.net1(x)
        y2 = self.net2(x)

        return torch.cat([y1,y2],1)

#定义损失函数
class loss_funcation():
    def __init__(self,model,punish=0.01):
        self.punish = punish
        self.model = model
    
    def __call__(self,y_hat,y,index=None):
        loss1 = (torch.sum((y-y_hat)**2)/(2*y.shape[0]))
        #读取参数
        param_list = []
        for name,parameter in self.model.named_parameters():
            param_list += list(parameter.flatten())  
        param = torch.tensor(param_list).reshape(2,-1)
        if index is None:
            loss2 = ((torch.sum((param[0]-param[1])**2))**0.5)*self.punish
        else:
            loss2 = ((torch.sum((param[0][index]-param[1][index])**2))**0.5)*self.punish
        return loss1+loss2

#导出参数序号
def param_index(model,rate):
    #读取参数
    param_list = []
    for name,parameter in model.named_parameters():
        param_list += list(parameter.flatten())  
    param = torch.tensor(param_list).reshape(2,-1)
    difference = abs(param[0]-param[1])
    index = torch.argsort(difference)[:int(difference.shape[0]*(1-rate))]
    return index

#归一化
def data_one(data):
    for i in range(data.shape[1]):
        ma = torch.max(data[:,i])
        mi = torch.min(data[:,i])
        cha = ma-mi
        # print(ma,mi,min(data[:,i]))
        for k in range(data.shape[0]):
            data[k,i] = (data[k,i]-mi)/cha
    return data

#创建模拟数据
def gen_data(w11,w12,w21,w22,m=1000,n=5,require=None,x_type="e"):
    if x_type == "n":
        x = torch.normal(0,1,(m,n))
    if x_type == "e":
        x = torch.tensor(np.random.exponential(10,(m,n)),dtype=torch.float32)
    if x_type == "u":
        x = torch.tensor(np.random.uniform(0,8,(m,n)),dtype=torch.float32)

    if require == None:
        noise = 0
        
    else:
        noise = torch.normal(0,1,(m,2))
    y1 = x@w11@w12
    # y1 = np.sin(x)@w11@w12
    y2 = x@w21@w22
    label = torch.cat([y1,y2],1)+noise
    return x,label

class gen_data_set(torch.utils.data.Dataset):
    #封装dataset
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self,index):
        x = self.x[index]
        y = self.y[index]
        return x,y
#训练模型
def train_model(model,loader_dict,optim,epoches,loss_f,index=None):
    for epoch in range(epoches):
        # print(f"epoch {epoch+1} / {epoches}")
        # print("-"*100)
        for phase in ["train","eval"]:
            epoch_loss = 0
            for input,label in loader_dict[phase]:
                optim.zero_grad()
                with torch.set_grad_enabled(phase=="train"):
                    output = model(input)
                    loss = loss_f(output,label,index)*input.shape[0]
                    if phase == "train":
                        loss.backward()
                        optim.step()
                    epoch_loss += loss
            print(f"{epoch}  {phase} : {epoch_loss/len(loader_dict[phase].dataset)}")

In [143]:
df = pd.read_excel(r"C:\Users\duduu\Desktop\deep learning\尧尧研创\2014-2022年8月空气污染数据(均值填充).xlsx")
from sklearn.model_selection import train_test_split
def data_one(data):
    for i in range(data.shape[1]):
        ma = torch.max(data[:,i])
        mi = torch.min(data[:,i])
        cha = ma-mi
        for k in range(data.shape[0]):
            data[k,i] = (data[k,i]-mi)/cha
    return data
x = df.loc[:,("SO2","CO","NO2","PM10")].values
y = df.loc[:,("PM2.5","O3_8h")].values
x = torch.tensor(x,dtype=torch.float32)
y = torch.tensor(y,dtype=torch.float32)
x = data_one(x)
y = data_one(y)
x_train,x_test,y_train,y_test = train_test_split(x,y)

input,label = gen_data(w11,w12,w21,w22,m=m,n=n,require=noise_require)
input1 = data_one(input.clone())
label1 = data_one(label.clone())
train_x,test_x,train_y,test_y = train_test_split(input1,label1,train_size=0.8)
train_dataset = gen_data_set(x_train,y_train)
eval_dataset = gen_data_set(x_test,y_test)
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=min_batch,shuffle=True)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset,batch_size=min_batch,shuffle=False)
dataloader_dict = {"train":train_dataloader,
                    "eval":eval_dataloader}

In [3]:
#各类超参数
m = 1000#样本量
n = 4#特征维度
min_batch = 64#小批量
w11 = torch.randn(n,3)#第一个任务第一层权重
# w12 = torch.randn(3,1)#第一个任务第二层权重
w21 = torch.normal(0,1,(n,3))
# w22 = torch.randn(3,1)
w12 = torch.tensor([[1.0],[2.0],[3.0]])
w22 = torch.tensor([[1.15],[2.15],[3.15]])
punish = 0.0005#惩罚项
lr = 0.005#学习率
noise_require = True #True

In [4]:
input,label = gen_data(w11,w12,w21,w22,m=m,n=n,require=noise_require)
input1 = data_one(input.clone())
label1 = data_one(label.clone())
train_x,test_x,train_y,test_y = train_test_split(input1,label1,train_size=0.8)
train_dataset = gen_data_set(train_x,train_y)
eval_dataset = gen_data_set(test_x,test_y)
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=min_batch,shuffle=True)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset,batch_size=min_batch,shuffle=False)
dataloader_dict = {"train":train_dataloader,
                    "eval":eval_dataloader}

In [194]:
#软参数
model = soft_net(n,4,3,1)
optim = torch.optim.Adam(model.parameters(),lr=lr)
loss_f = loss_funcation(model,punish)
train_model(model,dataloader_dict,optim,210,loss_f)

0  train : 0.03363516181707382
0  eval : 0.02923865243792534
1  train : 0.02927461639046669
1  eval : 0.026414060965180397
2  train : 0.027056138962507248
2  eval : 0.025089265778660774
3  train : 0.02614506334066391
3  eval : 0.0247429758310318
4  train : 0.025846095755696297
4  eval : 0.024739230051636696
5  train : 0.025839922949671745
5  eval : 0.02476973831653595
6  train : 0.025876276195049286
6  eval : 0.024757249280810356
7  train : 0.02585025131702423
7  eval : 0.024774976074695587
8  train : 0.02586258202791214
8  eval : 0.0247653741389513
9  train : 0.02584569901227951
9  eval : 0.02471121773123741
10  train : 0.025896960869431496
10  eval : 0.0247981995344162
11  train : 0.025843942537903786
11  eval : 0.02472580410540104
12  train : 0.025858068838715553
12  eval : 0.02474614605307579
13  train : 0.025850262492895126
13  eval : 0.0247260220348835
14  train : 0.02588540129363537
14  eval : 0.02475167065858841
15  train : 0.02585773542523384
15  eval : 0.02473161369562149
16 

In [193]:
#不加惩罚项
model = soft_net(n,4,3,1)
optim = torch.optim.Adam(model.parameters(),lr=lr)
loss_f = loss_funcation(model,punish=0)
train_model(model,dataloader_dict,optim,210,loss_f)

0  train : 0.03721237927675247
0  eval : 0.03058800660073757
1  train : 0.02920999564230442
1  eval : 0.025306904688477516
2  train : 0.02538616955280304
2  eval : 0.02346804179251194
3  train : 0.024313537403941154
3  eval : 0.023175787180662155
4  train : 0.024239614605903625
4  eval : 0.023217806592583656
5  train : 0.02426314912736416
5  eval : 0.02316111885011196
6  train : 0.024244174361228943
6  eval : 0.023143885657191277
7  train : 0.024236038327217102
7  eval : 0.023142872378230095
8  train : 0.02424433082342148
8  eval : 0.023138392716646194
9  train : 0.024238890036940575
9  eval : 0.023130517452955246
10  train : 0.024245744571089745
10  eval : 0.023165525868535042
11  train : 0.024238677695393562
11  eval : 0.02313639596104622
12  train : 0.024238277226686478
12  eval : 0.02313639409840107
13  train : 0.02423916570842266
13  eval : 0.023153994232416153
14  train : 0.024237580597400665
14  eval : 0.023138584569096565
15  train : 0.024243220686912537
15  eval : 0.0231390353

In [195]:
#自适应参数惩罚项
model = soft_net(n,4,3,1)
optim = torch.optim.Adam(model.parameters(),lr=lr)
loss_f = loss_funcation(model,punish)
train_model(model,dataloader_dict,optim,1,loss_f)
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    index = param_index(model,i)
    loss_f = loss_funcation(model,punish)
    train_model(model,dataloader_dict,optim,1,loss_f,index)

0  train : 0.02547510713338852
0  eval : 0.024396026507019997
0  train : 0.02512911707162857
0  eval : 0.024001827463507652
0  train : 0.024912415072321892
0  eval : 0.023794636130332947
0  train : 0.02476642094552517
0  eval : 0.02361229807138443
0  train : 0.02466581016778946
0  eval : 0.02357923798263073
0  train : 0.024541519582271576
0  eval : 0.02340228483080864
0  train : 0.024481646716594696
0  eval : 0.023395007476210594
0  train : 0.024365447461605072
0  eval : 0.023264586925506592
0  train : 0.024285711348056793
0  eval : 0.023152507841587067
0  train : 0.024272069334983826
0  eval : 0.023187315091490746
0  train : 0.02424723654985428
0  eval : 0.02313636615872383


In [156]:
model = soft_net(n,4,3,1)
optim = torch.optim.Adam(model.parameters(),lr=lr)
loss_f = loss_funcation(model,punish=punish)

In [157]:
# for _ in range(100):
train_model(model,dataloader_dict,optim,200,loss_f)

0  train : 0.0406477153301239
0  eval : 0.03538883104920387
1  train : 0.03131786361336708
1  eval : 0.027192501351237297
2  train : 0.02622252143919468
2  eval : 0.0227962676435709
3  train : 0.023968806490302086
3  eval : 0.020996563136577606
4  train : 0.02342749759554863
4  eval : 0.02031157724559307
5  train : 0.023259872570633888
5  eval : 0.020106781274080276
6  train : 0.023235242813825607
6  eval : 0.020126914605498314
7  train : 0.02323061227798462
7  eval : 0.020112695172429085
8  train : 0.023234773427248
8  eval : 0.020079219713807106
9  train : 0.023223914206027985
9  eval : 0.020111367106437683
10  train : 0.02322867140173912
10  eval : 0.02015507221221924
11  train : 0.023236894980072975
11  eval : 0.02012210339307785
12  train : 0.02322879247367382
12  eval : 0.020094886422157288
13  train : 0.023239493370056152
13  eval : 0.020174257457256317
14  train : 0.02323741838335991
14  eval : 0.02012120559811592
15  train : 0.023236529901623726
15  eval : 0.020104825496673584

In [176]:
index = param_index(model,1)

In [186]:
# for _ in range(20):
train_model(model,dataloader_dict,optim,10,loss_f,index)

0  train : 0.02287113666534424
0  eval : 0.019984418526291847
1  train : 0.022902486845850945
1  eval : 0.020153814926743507
2  train : 0.022845495492219925
2  eval : 0.020038355141878128
3  train : 0.02285665087401867
3  eval : 0.01998898573219776
4  train : 0.02285449579358101
4  eval : 0.020066820085048676
5  train : 0.022846484556794167
5  eval : 0.020126650109887123
6  train : 0.022850114852190018
6  eval : 0.02014332078397274
7  train : 0.02287016436457634
7  eval : 0.020136702805757523
8  train : 0.022860372439026833
8  eval : 0.020159833133220673
9  train : 0.02287468872964382
9  eval : 0.01996036432683468
